In [82]:
import pandas as pd
import numpy as np
import xarray as xr
import cdsapi 

# viz
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
from cartopy.io import shapereader
import plotly.graph_objects as go

import glob
import sys
import os

import geopandas as gpd
import rasterio
from rasterio.features import geometry_mask

# creds
sys.path.append('/Users/max/Deep_Sky')
from creds import CDS_UID, CDS_API_key

In [3]:

# point to local UNSEEN open dir to import retrieve and preprocess
sys.path.append('/Users/max/Deep_Sky/GitHub/UNSEEN-open-Deep-Sky')

import src.cdsretrieve as retrieve
import src.preprocess as preprocess

c = cdsapi.Client()

In [4]:
SEAS5_N_America = preprocess.merge_SEAS5(folder ='../../data/UNSEEN/wildfire/SEAS5/', target_months = [6,7,8])
SEAS5_N_America

Lead time: 05
4
3


<xarray.Dataset> Size: 401MB
Dimensions:    (leadtime: 3, time: 129, number: 51, latitude: 31, longitude: 41)
Coordinates:
  * longitude  (longitude) float32 164B -120.0 -119.0 -118.0 ... -81.0 -80.0
  * latitude   (latitude) float32 124B 50.0 49.0 48.0 47.0 ... 22.0 21.0 20.0
  * number     (number) int32 204B 0 1 2 3 4 5 6 7 8 ... 43 44 45 46 47 48 49 50
  * time       (time) datetime64[ns] 1kB 1981-06-01 1981-07-01 ... 2023-08-01
  * leadtime   (leadtime) int64 24B 2 3 4
Data variables:
    t2m        (leadtime, time, number, latitude, longitude) float64 201MB dask.array<chunksize=(1, 3, 51, 31, 41), meta=np.ndarray>
    d2m        (leadtime, time, number, latitude, longitude) float64 201MB dask.array<chunksize=(1, 3, 51, 31, 41), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-13 16:59:40 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [60]:
ERA5_N_America = xr.open_mfdataset('../../data/UNSEEN/wildfire/ERA5_new/ERA5_????.nc',combine='by_coords')
ERA5_N_America

<xarray.Dataset> Size: 3MB
Dimensions:    (time: 135, latitude: 31, longitude: 41)
Coordinates:
  * longitude  (longitude) float32 164B -120.0 -119.0 -118.0 ... -81.0 -80.0
  * latitude   (latitude) float32 124B 50.0 49.0 48.0 47.0 ... 22.0 21.0 20.0
  * time       (time) datetime64[ns] 1kB 1979-06-01 1979-07-01 ... 2023-08-01
Data variables:
    t2m        (time, latitude, longitude) float64 1MB dask.array<chunksize=(3, 31, 41), meta=np.ndarray>
    d2m        (time, latitude, longitude) float64 1MB dask.array<chunksize=(3, 31, 41), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-13 18:55:34 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [61]:
def season_mean(ds, years, calendar='standard'):
    # Make a DataArray with the number of days in each month, size = len(time)
    month_length = ds.time.dt.days_in_month

    # Calculate the weights by grouping by 'time.season'
    weights = month_length.groupby('time.year') / month_length.groupby('time.year').sum()

    # Test that the sum of the weights for each season is 1.0
    np.testing.assert_allclose(weights.groupby('time.year').sum().values, np.ones(years))

    # Calculate the weighted average
    return (ds * weights).groupby('time.year').sum(dim='time', min_count = 3)

In [62]:
ERA5_yearly = season_mean(ERA5_N_America, len(pd.to_datetime(ERA5_N_America.time.values).year.unique()))

In [7]:
# # Calculate the mean 't2m' value
mean_t2m = ERA5_yearly['t2m'].mean(['latitude','longitude'])

fig = go.Figure(data=go.Scatter(
    x=mean_t2m.year.values, 
    y=mean_t2m.values,
    mode='markers',
    marker=dict(
        size=6,
        color='LightSeaGreen',
    ),
    text=mean_t2m.year.values  # Display the year on hover
))

fig.update_layout(
    xaxis_title="Time",
    yaxis_title="Mean t2m",
    title="Mean temperature",
)

fig.show()

In [73]:
SEAS5_yearly = season_mean(SEAS5_N_America, len(pd.to_datetime(SEAS5_N_America.time.values).year.unique()))
SEAS5_yearly


<xarray.Dataset> Size: 134MB
Dimensions:    (year: 43, leadtime: 3, number: 51, latitude: 31, longitude: 41)
Coordinates:
  * longitude  (longitude) float32 164B -120.0 -119.0 -118.0 ... -81.0 -80.0
  * latitude   (latitude) float32 124B 50.0 49.0 48.0 47.0 ... 22.0 21.0 20.0
  * number     (number) int32 204B 0 1 2 3 4 5 6 7 8 ... 43 44 45 46 47 48 49 50
  * leadtime   (leadtime) int64 24B 2 3 4
  * year       (year) int64 344B 1981 1982 1983 1984 ... 2020 2021 2022 2023
Data variables:
    t2m        (year, leadtime, number, latitude, longitude) float64 67MB dask.array<chunksize=(1, 1, 51, 31, 41), meta=np.ndarray>
    d2m        (year, leadtime, number, latitude, longitude) float64 67MB dask.array<chunksize=(1, 1, 51, 31, 41), meta=np.ndarray>

In [9]:
# # Calculate the mean 'd2m' value
mean_d2m = SEAS5_yearly['d2m'].mean(['latitude','longitude', 'number', 'leadtime'])

fig = go.Figure(data=go.Scatter(
    x=mean_d2m.year.values, 
    y=mean_d2m.values,
    mode='markers',
    marker=dict(
        size=6,
        color='LightSeaGreen',
    ),
    text=mean_t2m.year.values  # Display the year on hover
))

fig.update_layout(
    xaxis_title="Time",
    yaxis_title="Mean d2m",
    title="Mean dewpoint temperature",
)

fig.show()

In [63]:
area_weights = np.cos(np.deg2rad(ERA5_N_America.latitude))

ERA5_events = (
    ERA5_yearly[['t2m','d2m']].sel(  
        longitude = slice(-125,-100),    
        latitude = slice(45,20)).          
    weighted(area_weights).
    mean(['longitude', 'latitude']) 
)

In [21]:
SEAS5_events = (
    SEAS5_yearly[['t2m','d2m']].sel(
        longitude = slice(-125,-100),   
        latitude = slice(45,20)).     
    weighted(area_weights).
    mean(['longitude', 'latitude'])
)

In [22]:
SEAS5_events.to_dataframe().to_csv('../../data/UNSEEN/wildfire/preprocessed/SEAS5_events.csv')
ERA5_events.to_dataframe().to_csv('../../data/UNSEEN/wildfire/preprocessed/ERA5_events.csv')

In [84]:
states_of_interest = ['MN', 'ND', 'SD', 'MT', 'WY', 'ID', 'WA', 'OR', 'CA', 'NV', 'UT', 'CO', 'AZ', 'NM', 'MI']



In [85]:
def get_shape(shapefile, state):
    shapefile = shapefile.to_crs("EPSG:4326")
    state_shape = shapefile[shapefile.STUSPS == state].geometry
    return state_shape

In [87]:
def mask_xr_dataset(xr_data, shape):
    transform = rasterio.transform.from_origin(west=xr_data.longitude.min().item(), north=xr_data.latitude.max().item(), xsize=1.0, ysize=1.0)
    mask = geometry_mask([geom for geom in shape.geometry], out_shape=(len(xr_data.latitude), len(xr_data.longitude)), transform=transform, invert=True)
    mask_xr = xr.DataArray(mask, dims=["latitude", "longitude"], coords={"latitude": xr_data.latitude, "longitude": xr_data.longitude})
    xr_data_masked = xr_data.where(mask_xr, drop=True)
    return xr_data_masked


def aggregate_annually(xr_data, vars):
    xr_data_yearly = season_mean(xr_data, len(pd.to_datetime(xr_data.time.values).year.unique()))
    area_weights = np.cos(np.deg2rad(xr_data.latitude))
    events = (
        xr_data_yearly[vars].       
        weighted(area_weights).
        mean(['longitude', 'latitude'])
    )
    return events

def store_state_xr_data(events, state, type):
    print(f'Storing {type} data for {state}')
    events.to_dataframe().to_csv(f'../../data/UNSEEN/wildfire/preprocessed/{type}_{state}_events.csv')



In [88]:
shapefile = gpd.read_file('../../data/US_State_Shapefiles/tl_2023_us_state.shp')

for state in states_of_interest:
    shape = get_shape(shapefile, state)
    state_data_era5 = mask_xr_dataset(ERA5_N_America, shape)
    state_data_seas5 = mask_xr_dataset(SEAS5_N_America, shape)
    era5_events = aggregate_annually(state_data_era5, ['t2m','d2m'])
    seas5_events = aggregate_annually(state_data_seas5, ['t2m','d2m'])
    store_state_xr_data(era5_events, state, 'ERA5')
    store_state_xr_data(seas5_events, state, 'SEAS5')

Storing ERA5 data for MN
Storing SEAS5 data for MN
Storing ERA5 data for ND
Storing SEAS5 data for ND
Storing ERA5 data for SD
Storing SEAS5 data for SD
Storing ERA5 data for MT
Storing SEAS5 data for MT
Storing ERA5 data for WY
Storing SEAS5 data for WY
Storing ERA5 data for ID
Storing SEAS5 data for ID
Storing ERA5 data for WA
Storing SEAS5 data for WA
Storing ERA5 data for OR
Storing SEAS5 data for OR
Storing ERA5 data for CA
Storing SEAS5 data for CA
Storing ERA5 data for NV
Storing SEAS5 data for NV
Storing ERA5 data for UT
Storing SEAS5 data for UT
Storing ERA5 data for CO
Storing SEAS5 data for CO
Storing ERA5 data for AZ
Storing SEAS5 data for AZ
Storing ERA5 data for NM
Storing SEAS5 data for NM
Storing ERA5 data for MI
Storing SEAS5 data for MI
